Nama:

In [1]:
print("Nama : Hanun Masitha Ramadhani")

Nama : Hanun Masitha Ramadhani


(1) Import dataset spam sms. Jadikan kolom 'v1' menjadi kolom target, dengan ham = 0, spam = 1.

In [75]:
import pandas as pd
df_sms = pd.read_csv('sms_spam_new.csv')
df_sms

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [94]:
y = df_sms['v1']
x = df_sms['v2']

In [95]:
y = y.map({'ham':0, 'spam':1})

In [96]:
y.value_counts()

0    4825
1     747
Name: v1, dtype: int64

(2) Periksa adanya missing value pada dataset, dan lakukan dropna apabila sedikit jumlahnya.

In [99]:
missing = []
persen = []

for i in df_sms.columns:
    missing.append(df_sms[i].isnull().sum())
    persen.append(df_sms[i].isnull().sum()/len(df_sms[i])*100)

proporsi_hilang = pd.DataFrame({
        'kolom' : df_sms.columns,
        'missing' : missing,
        'persen_missing' : persen
})
    
proporsi_hilang

,kolom,missing,persen_missing
0,v1,0,0.0
1,v2,0,0.0


(3) Lakukan train test split, test:test = 4:1, stratify = y.

In [100]:
# Data Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2
)

print("Jumlah Data Train : {0}".format(len(y_train)))
print("Jumlah Data Test : {0}".format(len(X_test)))

Jumlah Data Train : 4457
Jumlah Data Test : 1115


In [101]:
y_train.shape

(4457,)

(4) Lakukan cross-validation dengan pipeline dengan step: countvectorizer dan multinomialnb untuk menentukan nilai optimal untuk hyperparameter max_df dan max_features untuk countvectorizer; serta class_prior untuk multinomialnb. 

In [102]:
import warnings
warnings.filterwarnings("ignore")

In [103]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

param_grid = {'countvectorizer__max_features':[1000,2000,10000], 
              'multinomialnb__class_prior':[[0.5,0.1],[0.5,0.07]]}

pipeline = make_pipeline(CountVectorizer(stop_words='english'), 
                             MultinomialNB())
gscv = GridSearchCV(pipeline, param_grid)
gscv.fit(X_train, y_train)
best_param = gscv.best_params_

In [104]:
best_param

{'countvectorizer__max_features': 10000,
 'multinomialnb__class_prior': [0.5, 0.07]}

In [105]:
pipeline_def = make_pipeline(CountVectorizer(stop_words='english'), 
                             MultinomialNB())
pipeline_opt = make_pipeline(CountVectorizer(max_features=best_param['countvectorizer__max_features'], stop_words='english'), 
                             MultinomialNB(class_prior=best_param['multinomialnb__class_prior']))

pipeline_def.fit(X_train, y_train)
pipeline_opt.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=10000, stop_words='english')),
                ('multinomialnb', MultinomialNB(class_prior=[0.5, 0.07]))])

(5) Print classification report untuk trainset dan testset (default vs optimized). Apakah optimisasi meningkatkan performansi dari model?

In [108]:
from sklearn.metrics import classification_report
y_pred_train = pipeline_def.predict(X_train)
y_pred_test = pipeline_def.predict(X_test)

print(classification_report(y_train, y_pred_train, labels=[1,0]))

              precision    recall  f1-score   support

           1       0.98      0.98      0.98       591
           0       1.00      1.00      1.00      3866

    accuracy                           0.99      4457
   macro avg       0.99      0.99      0.99      4457
weighted avg       0.99      0.99      0.99      4457



In [109]:
y_pred_train = pipeline_opt.predict(X_train)
y_pred_test = pipeline_opt.predict(X_test)

print(classification_report(y_train, y_pred_train, labels=[1,0]))

              precision    recall  f1-score   support

           1       0.98      0.98      0.98       591
           0       1.00      1.00      1.00      3866

    accuracy                           0.99      4457
   macro avg       0.99      0.99      0.99      4457
weighted avg       0.99      0.99      0.99      4457



untuk hasil default dan optimasi hampir sama. hal ini dikarenakan hasil optimasi parameter mendapatkan hasil yang hampir sama dengan nilai default